In [1]:
library(itecv) # should be loaded first
library(magrittr)
library(foreach)
library(doFuture)
library(zeallot)
library(tidyverse)
library(rlearner)

plotsize = function (x,y) options(repr.plot.width=x, repr.plot.height=y)

registerDoFuture()
plan(cluster, workers = 4)

Loading required package: future
Loading required package: iterators
Loading required package: parallel

Attaching package: ‘zeallot’

The following objects are masked from ‘package:future’:

    %->%, %<-%

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
Warning message:
“package ‘dplyr’ was built under R version 3.5.1”── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ tidyr::extract()    masks magrittr::extract()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::set_names()  masks magrittr::set_names()
✖ purrr::when()       masks foreach::when()
Warning message:
“replacing previous import ‘caret::lift’ by ‘purrr::lift’ when loading ‘rlearner’”Warning message:
“replacing previous import 

In [44]:
model_specs = list(
    gbm = list(
        tune_grid = expand.grid(
            n.trees = seq(1,501,40), 
            interaction.depth=3, 
            shrinkage = 0.1, 
            n.minobsinnode=3),
        extra_args = list(
            verbose=F, 
            bag.fraction=0.5)),
    glmnet = list(
        tune_grid = expand.grid(
            alpha=c(0,0.5,1),
            lambda=exp(seq(-5,2,0.4))),
        extra_args = list()))

metrics = list(
    mse = bundle_mse,
    wmse = bundle_wmse,
#     dr_mse = bundle_dr_mse,
    r_objective = bundle_r_objective,
    match_mse = bundle_match_mse,
    trans_mse = bundle_trans_mse,
#     match_mdc = bundle_match_mdc,
#     trans_mdc = bundle_trans_mdc,
    ip_value = bundle_ip_value,
    dml_value = bundle_dml_value,
#     gain = bundle_gain,
#     c_benefit = bundle_c_benefit,
#     qini = bundle_qini,
    random = bundle_random)

replicates = 1:2
DGPs = powers_DGPs()[1:2] %>% list(1:2) %>% 
    pmap(function(dgp, id) {
        dgp$id = id
        dgp
    })

In [75]:
?unlist

In [76]:
c(itrain, ival, itest) %<-% make_indices(150, 50, 100)
cv_indices = function(itrain, ival, n_folds) {
    train_total = c(itrain, ival)
    n_train = length(train_total)
    splits = sample(train_total) %>%
        split(rep(1:n_folds, ceiling(n_train/n_folds)))
    icv = 1:n_folds %>% map(function(fold){
        list(
            train = splits[-fold] %>% unlist(use.names=F),
            val = splits[[fold]])
    })
}
icv = cv_indices(itrain, ival, 5)

In [80]:
itrain

[1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150

In [79]:
ival

[1] 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 [19] 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186
 [37] 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204
 [55] 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222
 [73] 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240
 [91] 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258
[109] 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276
[127] 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294
[145] 295 296 297 298 299 300

In [77]:
icv

[[1]]
[[1]]$train
  [1]  90 178  33 219 112 216 227 155 275 266 195 148 114  92 284  20  14  75
 [19] 292 121 129  60  74  30  84  55 297 279 243 224 263 169 276  70 256 182
 [37] 107  59 198 141 120  58 194 179 189 105 286  36   3  77 104 140  61 280
 [55]  73 109 214 163  62 259 221  41 154 128  23 145 134 254 118 157   9 212
 [73] 240 283 269 237 187 191 215  25 113 174 131 101 291  66 171 267 285 257
 [91]  97  69 156  29  48 115  22   1  88  38  46 116 164 205  86 228  67   2
[109] 188 136 103  96 168  78  13 238  26 213 248 102 242 111 173 207 119 234
[127] 147  89 192 230 293 202  10 209   6 271 239  52 232 160 153 287 193 144
[145] 137 130 299  15 231 108 273  99 133 123 281 172 290  63  57  80  94 218
[163]  34  18  51 126  19 258 150  83  31 127  49  45 220 201 250  50 264 226
[181]  72 146 186  64  43  28 229 265  21 260 223 180 255  40  53  76 206  87
[199]  85 122 151 274   7 159 277 298 158 300 252 132  91 161  35 196 165  65
[217] 125 210 138 288   5  82 183 197  71 200 124 235 247 249  11  44  12 268
[235] 270  95 142 143 162 139

[[1]]$val
 [1] 236  32  47 185 184 253 208 204 251 117  81  98 110 217 190 166  79 282 149
[20] 241 272 294  68  54 233 225 246 261 245 296  24 244  16 203 106 211  56 289
[39] 177  93 135 181 170 175 295   8  42  39 152 262 199 278  17 167   4 176 222
[58]  37  27 100


[[2]]
[[2]]$train
  [1] 236  32  47 185 184 253 208 204 251 117  81  98 110 217 190 166  79 282
 [19] 149 241 272 294  68  54 233 225 246 261 245 296  24 244  16 203 106 211
 [37]  56 289 177  93 135 181 170 175 295   8  42  39 152 262 199 278  17 167
 [55]   4 176 222  37  27 100 221  41 154 128  23 145 134 254 118 157   9 212
 [73] 240 283 269 237 187 191 215  25 113 174 131 101 291  66 171 267 285 257
 [91]  97  69 156  29  48 115  22   1  88  38  46 116 164 205  86 228  67   2
[109] 188 136 103  96 168  78  13 238  26 213 248 102 242 111 173 207 119 234
[127] 147  89 192 230 293 202  10 209   6 271 239  52 232 160 153 287 193 144
[145] 137 130 299  15 231 108 273  99 133 123 281 172 290  63  57  80  94 218
[163]  34  18  51 126  19 258 150  83  31 127  49  45 220 201 250  50 264 226
[181]  72 146 186  64  43  28 229 265  21 260 223 180 255  40  53  76 206  87
[199]  85 122 151 274   7 159 277 298 158 300 252 132  91 161  35 196 165  65
[217] 125 210 138 288   5  82 183 197  71 200 124 235 247 249  11  44  12 268
[235] 270  95 142 143 162 139

[[2]]$val
 [1]  90 178  33 219 112 216 227 155 275 266 195 148 114  92 284  20  14  75 292
[20] 121 129  60  74  30  84  55 297 279 243 224 263 169 276  70 256 182 107  59
[39] 198 141 120  58 194 179 189 105 286  36   3  77 104 140  61 280  73 109 214
[58] 163  62 259


[[3]]
[[3]]$train
  [1] 236  32  47 185 184 253 208 204 251 117  81  98 110 217 190 166  79 282
 [19] 149 241 272 294  68  54 233 225 246 261 245 296  24 244  16 203 106 211
 [37]  56 289 177  93 135 181 170 175 295   8  42  39 152 262 199 278  17 167
 [55]   4 176 222  37  27 100  90 178  33 219 112 216 227 155 275 266 195 148
 [73] 114  92 284  20  14  75 292 121 129  60  74  30  84  55 297 279 243 224
 [91] 263 169 276  70 256 182 107  59 198 141 120  58 194 179 189 105 286  36
[109]   3  77 104 140  61 280  73 109 214 163  62 259 242 111 173 207 119 234
[127] 147  89 192 230 293 202  10 209   6 271 239  52 232 160 153 287 193 144
[145] 137 130 299  15 231 108 273  99 133 123 281 172 290  63  57  80  94 218
[163]  34  18  51 126  19 258 150  83  31 127  49  45 220 201 250  50 264 226
[181]  72 146 186  64  43  28 229 265  21 260 223 180 255  40  53  76 206  87
[199]  85 122 151 274   7 159 277 298 158 300 252 132  91 161  35 196 165  65
[217] 125 210 138 288   5  82 183 197  71 200 124 235 247 249  11  44  12 268
[235] 270  95 142 143 162 139

[[3]]$val
 [1] 221  41 154 128  23 145 134 254 118 157   9 212 240 283 269 237 187 191 215
[20]  25 113 174 131 101 291  66 171 267 285 257  97  69 156  29  48 115  22   1
[39]  88  38  46 116 164 205  86 228  67   2 188 136 103  96 168  78  13 238  26
[58] 213 248 1

In [55]:
n_folds = 5
train_total = c(itrain, ival)
n_train = length(train_total)
splits = sample(train_total) %>%
    split(rep(1:n_folds, ceiling(n_train/n_folds)))

In [72]:
length(ival)

[1] 150

In [70]:
train_total %>% length

[1] 300

In [68]:
splits[-1] %>% unlist %>% length

[1] 240

In [7]:
library(rsample)

Loading required package: broom

Attaching package: ‘rsample’

The following object is masked from ‘package:tidyr’:

    fill



In [39]:
train_total = c(itrain, ival)
n_train = length(train_total) 
n_folds = 5

In [42]:
sample(train_total) %>% split(1:n_train>n_train/n_folds) %>% map(length)

$`FALSE`
[1] 40

$`TRUE`
[1] 160

In [8]:
DGP = DGPs[[1]]
data = create_data(DGP, length(c(itrain, ival, itest)))

In [3]:
learners = list(
     rlearner=rlearner_cv, 
     slearner=slearner_cv, 
     tlearner=tlearner_cv) 

In [4]:
run_sim = function(replicate, DGP, model_specs, learners, metrics, itrain, ival, itest) {
    data = create_data(DGP, length(c(itrain, ival, itest)))
    estimates = estimate_val_test(data, itrain, model_specs)
    val_bundle = learn_validation_auxiliaries(data, ival, model_specs, randomized=DGP$randomized)
    val_metrics = estimate_val_metrics(estimates, val_bundle, metrics, ival)
    test_metrics = calc_test_metrics(data, estimates, itest)
    
    learner_estimates = estimate_learner_test(data, learners, model_specs, itrain, ival, itest)
    
    learner_result = calc_test_metrics(data, learner_estimates, itest) %>%
        mutate(replicate=replicate, dgp=DGP$id)
    
    selection_result = inner_join(val_metrics, test_metrics, by="model") %>%
        mutate(replicate=replicate, dgp=DGP$id)
    
    list(selection_result, learner_result)
}

In [5]:
start_time <- Sys.time()

struct = foreach(replicate = replicates, 
        .combine = c,
        .export=c("model_specs", "learners", "metrics", "itrain", "ival", "itest", "run_sim"),
        .packages = (.packages())) %:%
    foreach(DGP = DGPs, 
#             .combine = list,
            .export=c("model_specs", "learners", "metrics", "itrain", "ival", "itest", "run_sim"),
            .packages = (.packages()))  %dopar%  {
        run_sim(replicate, DGP, model_specs, learners, metrics, itrain, ival, itest) 
    } %>% transpose() %>%
map(bind_rows)
struct[[1]] %>% write_csv("data/experimental_results_selection.csv")
struct[[2]] %>% write_csv("data/experimental_results_learner.csv")

end_time <- Sys.time()
end_time - start_time

Time difference of 2.593124 mins